In [1]:
#%%writefile makeTracks.py
def check_position_size(box):
    Xpos=1.*(box[0]>400) * 1.*(box[0]<1000)
    Ypos=1.*(box)[1]>50 * 1.*(box[1]<220)
    Size=1.*(box[2])>130
    return Xpos*Ypos*Size

def is_match(f1,f2):
    diff=max(abs(f1-f2))
    return diff<15

from numpy import *
from glob import glob
import pickle

for filename in glob('../output/*_Faces.pkl'):
    print '#'*50,'processing', filename
    Faces =pickle.load(open(filename,'r'))

    # remove faces whose parameters are too atypical
    GoodFaces=[]
    remove=0
    for F in Faces:
        L=[]
        for f in F:
            if check_position_size(f):
                L.append(f)
            else:
                remove+=1
        GoodFaces.append(L)
 
    print '\nremoved',remove,'atypical faces'
    # Organize Face detections into tracks
    Tracks=[]
    for i in range(len(GoodFaces)):
        print '\r',i,
        F=GoodFaces[i]

        for f in F:
            attached=False
            for track in Tracks:
                j,fprev=track[-1]
                if i-j<10 and is_match(f,fprev):
                    track.append((i,f))
                    attached=True
                    break
            if not attached:
                Tracks.append([(i,f)])
    all_tracks=len(Tracks)
    Tracks=[f for f in Tracks if len(f)>100]
    print '\nall tracks:',all_tracks,'long tracks:',len(Tracks)

    #calc some statistics of the tracks
    n=0
    for track in Tracks:
        gaps=0
        diffs=[]
        for i in range(1,len(track)):
            gap=track[i][0]-track[i-1][0]
            gaps+=gap-1
            diffs.append((track[i][1]-track[i-1][1])/gap)
        print n,track[0][0],gaps,len(track)
        print 'mean=',mean(stack([t[1] for t in track]),axis=0)[:3],
        print 'std of diffs=',std(stack(diffs),axis=0)[:3]
        n+=1

    #smooth the tracks
    k=2
    SmoothTracks=[]
    print '\n smoothing the tracks'
    print 'start end length'
    for track in Tracks:
        start=track[0][0]
        end=track[-1][0]
        length=end-start+1
        print start,end,length
        Track1=empty([length,5])
        Track1[:,:]=nan
        #put the entries in track into the array Track1, leaving nan values in the holes
        for i in range(len(track)):
            index=track[i][0]-start
            Track1[index,1:5]=track[i][1]

        Track2=empty([length,5])
        Track2[:,:]=nan
        for i in range(length):
            Track2[i][0]=start+i
            Track2[i][1:5]=nanmean(Track1[max(i-k,0):min(i+k,length),1:5],axis=0)

        Track3=array(Track2,dtype=int16)  

        SmoothTracks.append(Track3)
    pickle.dump(SmoothTracks,open(filename[:-9]+'Tracks.pkl','w'))
    break

################################################## processing ../output/'Homeland' Suffered A Major Intelligence Failure-9IE1mUL1erk_Faces.pkl

removed 8200 atypical faces
10223 
all tracks: 78 long tracks: 10
0 3 1 2031
mean= [ 712.69325455  130.0388971   145.81880847] std of diffs= [ 2.86356629  2.13889566  2.58099729]
1 2486 5 1219
mean= [ 724.56521739  149.72764561  145.41509434] std of diffs= [ 2.26379857  2.08124824  2.9059484 ]
2 3815 0 243
mean= [ 724.83950617  151.64197531  145.65843621] std of diffs= [ 2.40058462  1.98768492  3.14733922]
3 4304 0 120
mean= [ 768.76666667  114.09166667  146.025     ] std of diffs= [ 2.543513    2.15545809  2.20830999]
4 4673 0 276
mean= [ 725.80797101  120.80797101  147.70289855] std of diffs= [ 2.07054431  2.31292738  2.65797039]
5 4997 16 158
mean= [ 715.44303797  134.3164557   140.47468354] std of diffs= [ 2.95947448  3.53773352  3.29537668]
6 5171 10 623
mean= [ 697.31139647  135.83627608  144.34510433] std of diffs= [ 1.97703704  2.432463

/Users/yoavfreund/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


7532 8744 1213
8756 10071 1316


In [3]:
!ls -l ../output/19\ Days\ And\ Counting-JxQKYgw9G2k*.pkl

-rw-r--r--  1 yoavfreund  staff  735293 Jun  7 15:59 ../output/19 Days And Counting-JxQKYgw9G2k_Faces.pkl
-rw-r--r--  1 yoavfreund  staff  256327 Jun  9 08:31 ../output/19 Days And Counting-JxQKYgw9G2k_Tracks.pkl


In [3]:
len(Faces)

10224

In [4]:
Faces[:10]

[(),
 (),
 (),
 array([[568, 132, 154, 154]], dtype=int32),
 array([[563, 130, 161, 161]], dtype=int32),
 array([[568, 135, 150, 150]], dtype=int32),
 array([[571, 136, 147, 147]], dtype=int32),
 array([[570, 134, 148, 148]], dtype=int32),
 array([[567, 134, 149, 149]], dtype=int32),
 array([[567, 133, 149, 149]], dtype=int32)]

In [10]:
track=SmoothTracks[0]
track[:10]

array([[  3, 565, 131, 157, 157],
       [  4, 566, 132, 155, 155],
       [  5, 567, 133, 153, 153],
       [  6, 568, 133, 151, 151],
       [  7, 569, 134, 148, 148],
       [  8, 568, 134, 148, 148],
       [  9, 567, 133, 148, 148],
       [ 10, 566, 134, 147, 147],
       [ 11, 565, 134, 146, 146],
       [ 12, 564, 134, 146, 146]], dtype=int16)

In [2]:
stack?